Maxwell solver for Pec using direct formulation
=============================
**keys**: Maxwell double layer potential, PEC scattering, MoM, Neumann trace

In [ ]:
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=1, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(3)

In [ ]:
fesHCurl = HCurl(mesh, order=3, complex=True)
fesHDiv = HDivSurface(mesh, order=3, complex=True)
uHCurl,vHCurl = fesHCurl.TnT() # H(curl_Gamma) conform spaces; trial and test
uHDiv,vHDiv = fesHDiv.TnT() # H(div_Gamma) conform spaces; trial and test

print ("ndof HCurl = ", fesHCurl.ndof)
print ("ndof HDiv = ", fesHDiv.ndof)

In [ ]:
eps0 = 8.854e-12 
mu0 = 4*pi*1e-7
omega = 1.5e9
kappa = omega*sqrt(eps0*mu0)
print("kappa = ", kappa)

E_inc = CF((1,0,0))*exp( -1j * kappa * z )

n = specialcf.normal(3)
mR = GridFunction(fesHCurl) 
mR.Set( - Cross( Cross(n, E_inc), n) , definedon=mesh.Boundaries(".*"), dual=True) # Hcurl

In [ ]:
# new operators, i.e., V: Hdiv is TnT; K: Hdiv as test and HCurl as trial space with rotated evaluator   
with TaskManager(): 
    V = MaxwellSingleLayerPotentialOperatorNew(fesHDiv, kappa, 
                                            intorder=10, leafsize=40, eta=3., eps=1e-4, method="aca", testhmatrix=False)
    K = MaxwellDoubleLayerPotentialOperatorNew(fesHCurl, fesHDiv, kappa, 
                                            intorder=12, leafsize=40, eta=3., eps=1e-6, method="aca", testhmatrix=False)

In [ ]:
# solve direct formulation for j using new operators
j = GridFunction(fesHDiv) 
pre = BilinearForm( uHDiv.Trace() * vHDiv.Trace() *ds).Assemble().mat.Inverse(freedofs=fesHDiv.FreeDofs()) 
with TaskManager(): 
    M = BilinearForm( uHCurl.Trace() * vHDiv.Trace()* ds(bonus_intorder=3)).Assemble() # <Hcurl, Hdiv>  
    rhs = ( (0.5 * M.mat - K.mat ) * mR.vec).Evaluate() 
    CG(mat = V.mat, pre=pre, rhs = rhs, sol=j.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False)

# sove EFIE 
j_efie = GridFunction(fesHDiv) # Hdiv
rhs_efie = LinearForm( mR * vHDiv.Trace() *ds(bonus_intorder=3)).Assemble() 
CG(mat = V.mat, pre=pre, rhs = rhs_efie.vec, sol=j_efie.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False);
curlE_inc = CF( (0,-1j*kappa,0) ) *exp( -1j * kappa * z ) 

# check j: it must hold j =j_efie - j_inc 
j_inc = GridFunction(fesHDiv) 
j_inc.Set( Cross(n, curlE_inc), definedon=mesh.Boundaries(".*"), dual=True) # Hdiv 
j_test = GridFunction(fesHDiv) 
j_test.Set (j_efie - j_inc, definedon=mesh.Boundaries(".*"), dual=True) # consistent in Hdiv

Draw(Norm(j), mesh, draw_vol=False, order=3);
Draw(Norm(j_test), mesh, draw_vol=False, order=3);

In [ ]:
# orignal operators, i.e., test and trial space are also Hdiv realized as nxHCurl
with TaskManager(): 
    V2 = MaxwellSingleLayerPotentialOperator(fesHCurl, kappa, 
                                            intorder=10, leafsize=40, eta=3., eps=1e-4, method="aca", testhmatrix=False)
    K2 = MaxwellDoubleLayerPotentialOperator(fesHCurl, kappa, 
                                            intorder=10, leafsize=40, eta=3., eps=1e-4, method="aca", testhmatrix=False)

In [ ]:
# solve for Neumann trace j with original operators
j2 = GridFunction(fesHCurl) 
pre2 = BilinearForm( uHCurl.Trace() * vHCurl.Trace() *ds).Assemble().mat.Inverse(freedofs=fesHCurl.FreeDofs()) 
with TaskManager(): 
    M = BilinearForm( uHCurl.Trace() * Cross(n, vHCurl.Trace()) * ds(bonus_intorder=3)).Assemble() # <Hcurl, Hdiv> 
    rhs = ( ( 0.5 * M.mat - K2.mat ) * mR.vec).Evaluate()
    CG(mat = V2.mat, pre=pre2, rhs = rhs, sol=j2.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False) # 

# solve EFIE and check j
j_efie = GridFunction(fesHCurl) 
rhs_efie = LinearForm( mR * Cross(n, vHCurl.Trace()) *ds(bonus_intorder=3)).Assemble() 
CG(mat = V2.mat, pre=pre2, rhs = rhs_efie.vec, sol=j_efie.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False);

# use the EFIE solution to check j: it must hold j = j_efie - j_inc
curlE_inc = CF( (0,-1j*kappa,0) ) *exp( -1j * kappa * z ) 
j_inc = GridFunction(fesHCurl) 
j_inc.Set( Cross( Cross(n, curlE_inc), n), definedon=mesh.Boundaries(".*"), dual=True) # (n x nabla E)xn  to Hcurl
j_test = GridFunction(fesHCurl) 
j_test.Set (j_efie - j_inc, definedon=mesh.Boundaries(".*"), dual=True) 

Draw(Norm(j2), mesh, draw_vol=False, order=3);
Draw(Norm(j_test), mesh, draw_vol=False, order=3);